In [ ]:
import pandas as pd
import sqlalchemy as sa
import numpy as np

import bokeh.io as bio
from bokeh.plotting import figure as bfig
from bokeh.models import WMTSTileSource, ColumnDataSource, LinearColorMapper, HoverTool
from bokeh.tile_providers import CARTODBPOSITRON

### Generate Data

In [ ]:
###### make dataframe from Regents Database

engine = sa.create_engine('sqlite:///db/nycedudata.db')
meta = sa.MetaData()

regents_query = "SELECT Schools.dbn, Schools.school_name, Schools.latitude, Schools.longitude, \
                    Regents.school_year, Regents.test_name, Demo_Categories.demo_var, Regents.test_num, \
                    Regents.mean_score, Regents.num_lt_65, Regents.num_gt_65, Regents.num_gt_80 \
                FROM Schools \
                    INNER JOIN Regents on Regents.dbn = Schools.dbn \
                    INNER JOIN Demo_Categories ON Demo_Categories.demo_cat_id = Regents.school_demo_id;"

regents_df = pd.read_sql_query(regents_query, engine)

k = 6378137
regents_df['latitude'] = regents_df['latitude'].apply( lambda x: np.log(np.tan((90+float(x)) * np.pi/360.0))*k )
regents_df['longitude'] = regents_df['longitude'].apply( lambda x: float(x) * (k * np.pi/180) )
regents_df['mean_score'] = regents_df['mean_score'].apply( lambda x: float(x) )

int_cols = ['school_year','num_lt_65','test_num','num_gt_65','num_gt_80']
regents_df[int_cols] = regents_df[int_cols].applymap( lambda x: int(x) )

regents_df['pct_lt_65'] = regents_df['num_lt_65']/regents_df['test_num']*100
regents_df['pct_gt_65'] = regents_df['num_gt_65']/regents_df['test_num']*100
regents_df['pct_gt_80'] = regents_df['num_gt_80']/regents_df['test_num']*100

# regents_df.head()

### SET MAPPING PARAMETERS

In [ ]:
###### Set search parameters
test_stat = 'pct_gt_65'
demo_var = 'black'
test_name = 'common core algebra'
school_year = 2016

###### Set map parameters
padding = 10

south = np.min(regents_df['latitude']) - padding
north = np.max(regents_df['latitude']) + padding
west = np.min(regents_df['longitude']) - padding
east = np.max(regents_df['longitude']) + padding

spectrum_low = 0
spectrum_high = 100

In [ ]:
###### Refine dataframe based on search parameters

source_df = regents_df[regents_df['school_year'] == school_year]
source_df = source_df[source_df['demo_var'] == demo_var]
source_df = source_df[source_df['test_name'] == test_name]

source_df = source_df[['dbn','school_name','longitude','latitude',test_stat]]

# source_df.head()

data_source = ColumnDataSource(source_df)

In [6]:
###### Create map of data

p = bfig(tools='pan, wheel_zoom', x_range=(west,east), y_range=(south,north), 
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)

color_map = LinearColorMapper(palette='Magma256', low=spectrum_low, high=spectrum_high)

p.circle(x='longitude', y='latitude', source=data_source, fill_color={'field':test_stat, 'transform':color_map}, size=10)
p.add_tools(HoverTool(tooltips=[('DBN','@dbn'),('Name','@school_name'),('Score','@{0}'.format(test_stat))]))
bio.show(p)